# Identify all markdown files in the master folder which contain folders of files

Functions that are useful to iterate through each file

In [14]:
import os

In [24]:
# Function that yields one file path at a time, ignoring .DS_Store and non-image files
def get_files_one_by_one(directory: str) -> str:
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Skip .DS_Store file
            if file == '.DS_Store':
                continue
            # Include only .md files
            if file.lower().endswith(".md"):
                yield os.path.join(root, file)

# Function that gets all file paths in a list by calling get_files_one_by_one
def get_file_paths(directory: str) -> list:
    return list(get_files_one_by_one(directory))

def get_file_name_and_parent_folder(file_path: str) -> (str, str):
    file_name = os.path.basename(file_path)  # Extracts the file name
    parent_folder = os.path.basename(os.path.dirname(file_path))  # Extracts the parent folder
    return parent_folder, file_name

In [18]:
list_of_md_files = get_file_paths("../data/ocr_results_outputs/")
list_of_md_files[:5]

['../data/ocr_results_outputs/document_1020/document_1020.md',
 '../data/ocr_results_outputs/document_904/document_904.md',
 '../data/ocr_results_outputs/document_138/document_138.md',
 '../data/ocr_results_outputs/document_597/document_597.md',
 '../data/ocr_results_outputs/document_107/document_107.md']

# Detect large tables in each file

# Extract the table data

# Save it as a CSV file 

# Reference the table in the Markdown